In [276]:
import numpy as np

In [277]:
def func1(x):
    x = np.array(x)
    i = np.array(range(1,len(x)+1))
    result = 1/40*np.sum(x**2) + 1 - np.prod(np.cos(x/i))
    
    return result

In [278]:
# Zmieniam znak przy 20 na koncu równania (z +20 na -20)
def func2(x):
    x = np.array(x)
    n = len(x)
    result_1 = 20 * np.exp(-0.2 * np.sqrt(1/n * np.sum(x**2)))
    result_2 = - np.exp(1/n * np.sum(np.cos(2*np.pi*x))) - 20 + np.e
    return result_1 + result_2

Wrzuciłem funkcje w wolframa:

https://www.wolframalpha.com/input/?i=20*e%28-0.2*sqrt%28x%5E2%29%29+-+exp%28cos%282*pi*x%29%29+%2B+20+%2B+e

Wygląda na to, że powinno też być max, a nie min ;)

In [279]:
def initial_pop_generation(x, n, start_wth_zeros=False, min_value_n=None,
                           max_value_n=None):
    '''
    x : int
        parzysta liczba osobników
    n : int
        parzysta liczba genów
    start_wth_zeros : bool
        True - początkowe cechy osobnika równ zeru
        False - losowo dobrane cechy początkowe
    '''
    population = []
    if start_wth_zeros:
        for i in range(x):
            individual = np.array([0 for i in range(0, n)])
            population.append(individual)
    else:
        if (min_value_n == None or max_value_n == None):
            raise Exception('Set min&max value for n or set start_with_zeros to True')
        
        for i in range(x):
            individual = np.array([np.random.randint(min_value_n, max_value_n) for i in range(0, n)])
            population.append(individual)

    return population

In [280]:
def crossing(indiv1, indiv2,n):
    '''
    indiv1

    indiv2
    '''
    #losowanie locus podzialu
    locus = np.random.randint(0, n)
    embryo1 = np.concatenate((indiv1[:locus],indiv2[locus:]))
    embryo2 = np.concatenate((indiv2[:locus],indiv1[locus:]))

    return embryo1, embryo2

In [281]:
def mutate(individual, p_mutate):
    '''
    individuals : [int]
        list of the genes of the individual
    '''

    for i in range(len(individual)):
        r = np.random.rand()
        if r < p_mutate/2:
            individual[i] -= 1
        elif r < p_mutate:
            individual[i] += 1
        
    return individual
        

In [285]:
def optimize(population, x, n, function, min_or_max,end_optim, p_mutate=0.2):
    '''
    population : list of numpy arrays
        zbiór wszystkich osobników
    '''
    best_individual_scores = []
    iteration = 0
    while True:
        iteration += 1
        #krzyzowanie i mutacja
        embryons = []
        for i in range(0, x, 2):
            crossing_result = crossing(population[i], population[i+1], n)
            for embryon in crossing_result:
                population.append(mutate(embryon,p_mutate))
        scores = []
        for individual in population:
            score = function(individual)
            scores.append(score)
            
        if min_or_max == 'max':
            reverse=True
        elif min_or_max == 'min':
            reverse=False
        else:
            raise Exception('min_or_max parameter invalid. Type "min" or "max".')
            
        population_score = sorted(zip(population, scores), key=lambda x:x[1], reverse=reverse)
        best_individual_gen = population_score[0][0]
        best_individual_score = population_score[0][1]
        
        best_individual_scores.append(best_individual_score)
            
        sorted_population, _ = zip(*population_score)
        population = list(sorted_population[:x])
        
        print(f'Iteration: {iteration} | Best score: {best_individual_score}')
        
        # zakończ jeśli nie ma poprawy w 100 ostatnich iteracjach (?)
        if len(set(best_individual_scores[-end_optim:])) == 1 and iteration > 10:
            break
        
        
    return best_individual_gen, best_individual_score
        